<a href="https://colab.research.google.com/github/siddharth-69/hello-world/blob/master/Copy_of_Untitled40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym

In [0]:
#load_the_libraries
import numpy as np
import tensorflow as tf
import math
import cPickle as pickle
import matplotlib.pyplot as plt

from modelAny import *

from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import rnn
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import embedding_ops
import gym

In [0]:
env=gym.make('CartPole-v0')

In [0]:
#setting_the_hyperparameters
H=8 #no._of_hidden_layer_neurons
lr=0.01
gamma=0.99
decay_rate=0.99 #for_RMS_prop_optimizer
resume=False #resume_from_previous_checkpoint

model_bs=3
real_bs=3

#imput_dimensionality
D=4

In [0]:
#policy_network
tf.reset_default_graph()
observations=tf.placeholder(tf.float32,[None,4],name='input_x')
W1=tf.get_variable("W1",shape=[4,H],tf.contrib.layers.xavier_initializer())
layer1=tf.nn.relu(tf.matmul(observations,W1))
W2=tf.get_variable("W2",shape=[H,1],tf.contrib.layers.xavier_initializer())
score=tf.nn.relu(tf.matmul(layer1,W2))
probability=tf.nn.sigmoid(score)

tvar=tf.trainable_variables()
input_y=tf.placeholder(tf.float32,[None,1],name="input_y")
advantages=tf.placeholder(tf.float32,name="reward_signal")
adam=tf.train.AdamOptimizer(learning_rate=lr)
W1_grad=tf.placeholder(tf.float32,name="batch_grad1")
W2_grad=tf.placeholder(tf.float32,name="batch_grad2")
batch_grad=[w1_grad,w2_grad]
loglik=tf.log(input_y*(input_y-probability)+(1-input_y)*(input_y+probability))
loss=-tf.reduce_mean(loglik*advantages)
new_grads=tf.gradients(loss,tvar)
update_grads=adam.apply_gradients(zip(batch_grad,tvar))

In [0]:
#here_we_implement_the_model_networks
mH=256 #model_layer_size
input_data=tf.placeholder(tf.float32,[None,5])
with tf.variable_scope('rnnlm'):
  softmax_w=tf.get_variable("softmax_w",[mH,50])
  softmax_b=tf.get_variable("softmax_b",[50])
  
previous_state=tf.placeholder(tf.float32,[None,5],name="prevous_state")
W1M=tf.get_variable("W1M",shape=[5,mH],initializer=tf.contrib.layers.xavier_initializer())
B1M=tf.Variable(tf.zeros([mH]),name="B1M")
layer1M=tf.nn.relu(tf.matmul(previous_state,W1M)+B1M)
W2M=tf.get_variable("W2M",shape=[mH,mH],initializer=tf.contrib.layers.xavier_initializer())
B2M=tf.Variable(tf.zeros([mH]),name="B2M")
layer2M=tf.nn.relu(tf.matmul(layer1M,W2M)+B2M)

wO=tf.get_variable("wO",shape=[mH,4],initializer=tf.contrib.layers.xavier_initializer())
wR=tf.get_variable("wR",shape=[mH,1],initializer=tf.contrib.layers.xavier_initializer())
wD=tf.get_variable("wD",shape=[mH,1],initializer=tf.contrib.layers.xavier_initializer())

bO=tf.Variable(tf.zeros([4]),name="bO")
bR=tf.Variable(tf.zeros([1]),name="bR")
bD=tf.Variable(tf.zeros([1]),name="bD")

predicted_observation=tf.matmul(layer2M,wO,name="predicted_observation")+bO
predicted_reward=tf.matmul(layer2M,wR,name="predicted_reward")+bR
predicted_done=tf.sigmoid(tf.matmul(layer2M,wD,name="predicted_done")+bD)

true_observation=tf.placeholder(tf.float32,[None,4],name="true_observation")
true_reward=tf.placeholder(tf.float32,[None,1],name="true_reward")
true_done=tf.placeholder(tf.float32,[None,1],name="true_done")

predicted_state=tf.concat(1,[predicted_observation,predicted_reward,predicted_done])

observation_loss=tf.square(true_observation-predicted_observation)

reward_loss=tf.square(true_reward-predicted_reward)

done_loss=tf.mul(predicted_done,true_done)+tf.mul(1-predicted_done,1-true_done)
done_loss=-tf.log(done_loss)

model_loss=tf.reduce_mean(observation_loss+reward_loss+done_loss)

adam=tf.train.AdamOptimizer(learning_rate=lr)
update_model=adam.minimize(model_loss)

In [0]:
#helper_functions

def reset_gradbuffer(gradbuffer):
  for ix,grad in enumerate(gradbuffer):
    gradbuffer[ix]=0
  return gradbuffer

def discount_rewards(r):
  discounted_r=np.zeros_like(r)
  running_ad=0
  for t in reversed(range(0,r.size)):
    running_ad=(running_ad*gamma)+r[t]
    discounted_r[t]=running_ad
  return discounted_r

#this_model_uses_our_model_to_produce_a_new_state_given_an_old_state_and_action.
def step_model(sess,xs,action):
  to_feed=np.reshape(np.hstack(xs[-1][0],np.array(action)),[1,5])
  my_predict=sess.run([predicted_state],feed_dict={previous_state:to_feed})
  reward=my_predict[0][:,4]
  observation=my_predict[0][:,0:4]
  observation[:,0]=np.clip(observation[:,0],-2.4,2.4)
  observation[:,2]=np.clip(observation[:,2],-0.4,0.4)
  doneP=np.clip(my_predict[0][:,5],0,1)
  if doneP>0.1 or len(xs)>=300:
    done=True
  else:
    done=False
  return observation,reward,done

In [0]:
#training_the_policy_and_the_model

xs,drs,ys,ds=[],[],[],[]
running_reward=None
reward_sum=0
episode_number=1
real_episodes=1
init=tf.initialize_all_variables()
batch_size=real_bs

draw_from_model=False #when_set_to_true_....will_use_the_model_for_observation

train_the_model=True 
train_the_policy=False

switch_point=1

#launch_the_graph

with tf.Session() as sess:
  rendering=False
  sess.run(init)
  observation=env.reset()
  x=observation
  grad_buffer=sess.run(tvar)
  grad_buffer=reset_gradbuffer(grad_buffer)
  while episode_number<=5000:
    #start_displaying_environment_once_performance_is_acceptably_high
    if (reward_sum/batch_size >150 and draw_from_model==False) or rendering==True:
      #HERE_WE_DISPLAY_THE_ENVIRONMENT_WRITE_YOUR_OWN_CODE
    
    
    x=np.reshape(observation,[1,4])
    tfprob=sess.run([probability],feed_dict={observations:x})
    
    action=1 if np.random.uniform()<tfprob else 0
    #record_the_intermediates_required_for_back_prop
    
    y=1 if action==0 else 0
    ys.append(y)
    
    #step_the_model_or_real_environment_and_get_new_measurements
    if draw_from_model==False:
      observation,reward,done,info=env.step(action)
    else:
      observation,reward,done=step_model(sess,xs,action)
    
    reward_sum+=reward
    
    ds.append(done)
    drs.append(reward)  #record_reward...this_is_done_after_we_call_step_to_get_reward_for_previous_action
    
    if done:
      if draw_from_model==False:
        real_episode+=1
      episode_number+=1
      
      #stack_together_all_inputs,hidden_states,action_gradients_and_reward_for_this_episode
      epx=np.vstack(xs)
      epy=np.vstack(ys)
      epr=np.vstack(drs)
      epd=np.vstack(ds)
      
      #reset_array_memory
      xs,ys,ds,drs=[],[],[],[]
      
      if train_the_model==True:
        actions=np.array([np.abs(y-1) for y in epy][:-1])
        state_prevs=epx[:-1,:]
        state_prevs=np.hstack([state_prevs,actions])
        state_nexts=epx[1:,:]
        rewards=np.array(epr[1:,:])
        dones=np.array(epd[1:,:])
        state_nextsALL=np.hstack([state_nexts,rewards,dones])
        
        feed_dict={previous_state:state_prevs,true_observation:state_nexts,true_done:dones,true_reward:rewards}
        
        loss,pState,_=sess.run([model_loss,predicted_state,update_model],feed_dict)
        
      if train_the_policy==True:
        
        discounted_epr=discount_rewards(epr).astype('float32')
        discounted_epr-=np.mean(discounted_epr)
        discounted_epr/=np.std(discounted_epr)
        tgrad=sess.run(new_grads,feed_dict={observations:epx,input_y:epy,advantages:discounted_epr})
        
        #if_gradients_become_too_large_end_training_process
        
        if np.sum(tgrad[0]==tgrad[0])==0:
          break
        for ix,grad in enumerate(tgrad):
          grad_buffer[ix]+=grad
        
      if switch_point+batch_size==episode_number:
        switch_point=episode_number
        if train_the_policy==True:
          sess.run(update_grads,feed_dict={W1_grad:grad_buffer[0],W2_grad:grad_buffer[1]})
          grad_buffer=reset_gradbuffer(grad_buffer)
          running_reward=reward_sum if running_reward is None else ((0.99*running_reward)+(0.01*reward_sum))
        if draw_from_model==False:
          print('World Perf: Episode %f. Reward %f. action: %f. mean reward %f.'%(real_episodes,reward_sum/real_bs,action, running_reward/real_bs))
          
          if reward_sum/batch_size>200:
            break
        
        reward_sum=0
        
        #once_the_model_has_been_trained_on_100_episodes_we_start_alternating_between_training_the_policy_from_the_model_and_training_the_model_from_the_reeal_environment
        
        if episode_number>100:
          draw_from_model=not draw_from_model
          train_the_model=not train_the_model
          train_the_policy=not train_the_policy
          
      if draw_from_model==True:
        observation=np.random.uniform(-0.1,0.1,[4]) #generate_a_resonable_starting_point
        batch_size=model_bs
      else:
        observation=env.reset()
        batch_size=real_bs
    
print(real_episodes)